In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Importing Necessary Modules** 

In [44]:
from sklearn.dummy import DummyRegressor 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
import numpy as np
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, GradientBoostingRegressor, HistGradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn.ensemble import AdaBoostRegressor
from sklearn.metrics import r2_score
from sklearn.linear_model import SGDRegressor, LinearRegression, LogisticRegression, Lasso, Ridge
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.svm import SVC, SVR
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import PolynomialFeatures

# **Loading Data**

In [5]:
df = pd.read_csv("/kaggle/input/engage-2-value-from-clicks-to-conversions/train_data.csv") 
x = df.drop("purchaseValue", axis=1) 
y = df['purchaseValue'] 
"""
model = DummyRegressor().fit(x,y) 
X_test = pd.read_csv("/kaggle/input/engage-2-value-from-clicks-to-conversions/test_data.csv") 
y_pred=model.predict(X_test) 
submission = pd.DataFrame({"id": range(0,X_test.shape[0]), "purchaseValue": y_pred}) 
submission.to_csv('submission.csv',index=False)
df["new_visits"]
"""

'\nmodel = DummyRegressor().fit(x,y) \nX_test = pd.read_csv("/kaggle/input/engage-2-value-from-clicks-to-conversions/test_data.csv") \ny_pred=model.predict(X_test) \nsubmission = pd.DataFrame({"id": range(0,X_test.shape[0]), "purchaseValue": y_pred}) \nsubmission.to_csv(\'submission.csv\',index=False)\ndf["new_visits"]\n'

# **Exploratory Data Analysis**

## **Data Types of different columns**

In [ ]:
print("Details about every colum: ")
df.info()

## **Descriptive statistics of numerical columns**

In [ ]:
print("Statistical summary about every column: ")
df.describe()

## **Null/Missing values in every column**

In [ ]:
print("Total number of null values in every column: ")
df.isnull().sum()

## **Number of unique values in every column**

In [ ]:
print("Total number of unique values in every column: ")
df.nunique()

In [ ]:
df.frame.hist(figsize=(12, 10), bins=30, edgecolor="black")
plt.subplots_adjust(hspace=0.7, wspace=0.4)

## **Filtering column names with the data type of the column**

In [6]:
cat = x.select_dtypes(include=["object", "bool"]).columns.tolist()
num = x.select_dtypes(include=["int64", "float64"]).columns.tolist()
bol = x.select_dtypes(include=["float64"]).columns.tolist()
print(cat, num)

['trafficSource.isTrueDirect', 'browser', 'device.screenResolution', 'trafficSource.adContent', 'trafficSource.keyword', 'screenSize', 'geoCluster', 'trafficSource.adwordsClickInfo.slot', 'device.mobileDeviceBranding', 'device.mobileInputSelector', 'trafficSource.campaign', 'device.mobileDeviceMarketingName', 'geoNetwork.networkDomain', 'device.operatingSystemVersion', 'device.flashVersion', 'geoNetwork.region', 'trafficSource', 'geoNetwork.networkLocation', 'os', 'geoNetwork.subContinent', 'trafficSource.medium', 'trafficSource.adwordsClickInfo.isVideoAd', 'browserMajor', 'locationCountry', 'device.browserSize', 'trafficSource.adwordsClickInfo.adNetworkType', 'socialEngagementType', 'geoNetwork.city', 'geoNetwork.metro', 'device.mobileDeviceModel', 'trafficSource.referralPath', 'device.language', 'deviceType', 'userChannel', 'device.browserVersion', 'device.screenColors', 'geoNetwork.continent', 'device.isMobile'] ['userId', 'gclIdPresent', 'sessionNumber', 'totals.visits', 'sessionId

# **Exploratory Data Analysis - Milestone - 1**

In [ ]:
df1 = x2[x2["totals.bounces"] == 0.0]
df1 = df1["pageViews"]
t = df1.index.tolist()
df3 = y.loc[t]
df1["purchaseValue"] = df3
d = df1.corr(df3)
print("Correlation Between purchaseValue and pageViews: ", d)
f = x.groupby("geoNetwork.continent")["totalHits"].mean()
print(f)
b = y[y == 0].index.tolist()
c = y[y != 0].index.tolist()
g = x.loc[b]
h = x.loc[c]
print(g.groupby("trafficSource.medium")["trafficSource.medium"].count())
print(g.groupby(["os", "browser"]).size().sort_values(ascending = False))
print(h["totalHits"].mean() / g["totalHits"].mean())
print(x[x["trafficSource.medium"] == "organic"]["trafficSource.medium"].count() / x["trafficSource.medium"].count())
print(g.groupby(["trafficSource.referralPath"]).size().sort_values(ascending = False))
df4 = df[["os", "purchaseValue"]]
print(df4.groupby("os")["purchaseValue"].mean())
df5 = (df.groupby("userId")["sessionId"].count().reset_index(name="mean"))
print(df5["mean"].mean())
df6 = df["userId"].unique()
print(len(df6))

# **Data Pre-Processing - EDA - Removing nuisance features which have only 1 unique value and classifying features correctly as numerical and categorical variables**


### This is done after analyzing that, the variables having only 1 unique value acts as nuisance varibale and doesn't contribute anything to the model. So, It is removed.Variables such as sessionId, userId, date are categorical in nature and should be transported from numerical type to categroical data. As the values of these variables are numbers, it is grouped into numerical data by default


In [7]:
print(x.shape)
for i in x.columns:
    if x[i].nunique() == 2:
        if np.nan in list(x[i].unique()):
            x = x.drop(i, axis=1)
    elif x[i].nunique() == 1:
        x = x.drop(i, axis=1)
x.shape

(116023, 51)


(116023, 29)

In [ ]:
print(len(x2.columns))
boolean = []
for i in x2.columns:
    if x2[i].nunique() == 1:
        x2 = x2.drop(i, axis=1)
    elif x2[i].nunique() == 2:
        boolean.append(i)
print(len(x2.columns))
print(x2.nunique())

In [8]:
cat1 = []
num1 = []
for i in x.columns:
    if i in num:
        num1.append(i)
    elif i in cat+bol:
        cat1.append(i)

In [9]:
len(num1) + len(cat1)

29

In [10]:
try:
    if "sessionId" in num1:
        num1.remove("sessionId")
        if "sessionId" not in cat1:
            cat1.append("sessionId")
    if "userId" in num1:
        num1.remove("userId")
        if "userId" not in cat1:
            cat1.append("userId")
    if "date" in num1:
        num1.remove("date")
        if "date" not in cat1:
            cat1.append("date")
    for i in bol:
        num1.remove(i)
        if i not in cat1:
            cat1.append(i)
except ValueError:
    pass
num1

['gclIdPresent', 'sessionNumber', 'totalHits', 'sessionStart']

In [11]:
print(num1)
print(cat1)
print(bol)
print(len(num1)+len(cat1))

['gclIdPresent', 'sessionNumber', 'totalHits', 'sessionStart']
['browser', 'trafficSource.adContent', 'trafficSource.keyword', 'geoCluster', 'trafficSource.adwordsClickInfo.slot', 'trafficSource.campaign', 'geoNetwork.networkDomain', 'geoNetwork.region', 'trafficSource', 'os', 'geoNetwork.subContinent', 'trafficSource.medium', 'locationCountry', 'geoNetwork.city', 'geoNetwork.metro', 'trafficSource.referralPath', 'deviceType', 'userChannel', 'geoNetwork.continent', 'device.isMobile', 'sessionId', 'userId', 'date', 'trafficSource.adwordsClickInfo.page', 'pageViews']
['trafficSource.adwordsClickInfo.page', 'pageViews', 'totals.bounces', 'new_visits']
29


# **EDA - Plots of the variables**

In [ ]:
sns.set_style("dark")
sns.set_context("notebook")

In [ ]:
plt.figure(figsize=(len(num1)*3, len(num1)*3))
for i in range(len(num1)):
    plt.subplot(len(num1), 1, i+1)
    sns.histplot(x=x2[(num1)[i]], kde=True)
    plt.xlim(right = float(x2[num1[i]].max()))
    plt.title((num1)[i])
plt.tight_layout()
plt.show()

In [ ]:
sns.pairplot(x2[num1], height=2)
plt.show()

In [ ]:
print(cat1)
plt.figure(figsize=(len(cat1), len(cat1)*3))
for i in range(len(cat1)):
    plt.subplot(len(cat1), 2, i+1)
    plt.xticks(rotation=45, fontsize=8)
    sns.countplot(x=x2[(cat1)[i]])
    plt.title((cat1)[i])
plt.tight_layout()
plt.show()

In [ ]:
plt.title("HeatMap between Categorical Variables")
plt.figure(figsize=(100, 100))
"""
sns.heatmap(pd.crosstab(x2["browser"], x2["geoNetwork.continent"]), annot=True, cmap="YlGnBu")
plt.show()
"""
for i in cat1:
    for j in cat1:
        sns.heatmap(pd.crosstab(x2[i], x2[j]), annot=True, cmap="YlGnBu")
plt.show()


In [ ]:
"""

# Method - 1 of Visualising Variables

sns.countplot(x=x2["geoNetwork.continent"])
plt.xticks(rotation=45, fontsize=8)
plt.show()
def plot_data(x1, t):
    if t == "num":
        sns.histplot(x=x2[x1])
    elif t == "obj":
        sns.countplot(x=x2[x1])
    plt.xticks(rotation=45, fontsize=5)
    plt.show()
for i in num:
    print("Plot of ", i, " variable")
    plot_data(i, "num")
for i in cat + bol:
    print("Plot of ", i, " variable")
    plot_data(i, "obj")
   
"""

# **Filling Missing Values - Imputation**

## Imputation Method - 1

In [12]:
pipe_impute = ColumnTransformer(transformers=[
    ("num", SimpleImputer(strategy="median"), num1),
    ("cat", SimpleImputer(strategy='most_frequent'), cat1),
    #("bol", SimpleImputer(strategy='constant', fill_value=0), bol)
])
x1 = pipe_impute.fit_transform(x)
x2 = pd.DataFrame(x1, columns=num1+cat1)
x2

,gclIdPresent,sessionNumber,totalHits,sessionStart,browser,trafficSource.adContent,trafficSource.keyword,geoCluster,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,...,trafficSource.referralPath,deviceType,userChannel,geoNetwork.continent,device.isMobile,sessionId,userId,date,trafficSource.adwordsClickInfo.page,pageViews
0,0.0,1.0,1.0,1500100799.0,Edge,Google Merchandise Store,(not provided),Region_2,Top,(not set),...,/intl/hr/yt/about/,desktop,Social,Americas,False,1500100799,61421,20170714,1.0,1.0
1,0.0,1.0,1.0,1495262065.0,Chrome,Google Merchandise Store,(not provided),Region_3,Top,(not set),...,/,desktop,Direct,Americas,False,1495262065,72287,20170519,1.0,1.0
2,0.0,2.0,6.0,1508510328.0,Chrome,Google Merchandise Store,(not provided),Region_2,Top,(not set),...,/,desktop,Organic Search,Europe,False,1508510328,25180,20171020,1.0,6.0
3,0.0,1.0,1.0,1483431838.0,Internet Explorer,Google Merchandise Store,(not provided),Region_4,Top,(not set),...,/yt/about/ja/,desktop,Social,Asia,False,1483431838,41295,20170103,1.0,1.0
4,0.0,1.0,66.0,1475804633.0,Chrome,Google Merchandise Store,(not provided),Region_3,Top,(not set),...,/,desktop,Direct,Americas,False,1475804633,113697,20161006,1.0,54.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116018,0.0,1.0,28.0,1500318402.0,Chrome,Google Merchandise Store,(not provided),Region_3,Top,(not set),...,/,desktop,Referral,Americas,False,1500318402,109014,20170717,1.0,26.0
116019,0.0,1.0,1.0,1478624150.0,Chrome,Google Merchandise Store,(not provided),Region_5,Top,(not set),...,/,desktop,Direct,Americas,False,1478624150,66111,20161108,1.0,1.0
116020,0.0,2.0,1.0,1470384216.0,Chrome,Google Merchandise Store,(not provided),Region_1,Top,(not set),...,/google-analytics-launches-public-demo-account...,desktop,Referral,Asia,False,1470384216,97614,20160805,1.0,1.0
116021,0.0,1.0,2.0,1506953297.0,Chrome,Google Merchandise Store,(not provided),Region_5,Top,(not set),...,/,desktop,Organic Search,Americas,False,1506953297,71050,20171002,1.0,2.0


## Train-Test Split
### Taking 20% Data as Test Data

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x2, y, test_size=0.2, random_state=42)
x_train

,gclIdPresent,sessionNumber,totalHits,sessionStart,browser,trafficSource.adContent,trafficSource.keyword,geoCluster,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,...,trafficSource.referralPath,deviceType,userChannel,geoNetwork.continent,device.isMobile,sessionId,userId,date,trafficSource.adwordsClickInfo.page,pageViews
44030,0.0,1.0,1.0,1506980371.0,Chrome,Google Merchandise Store,(not provided),Region_3,Top,(not set),...,/,mobile,Organic Search,Americas,True,1506980371,97943,20171002,1.0,1.0
71236,0.0,34.0,64.0,1479492062.0,Chrome,Google Merchandise Store,(not provided),Region_2,Top,(not set),...,/,desktop,Referral,Americas,False,1479492062,86785,20161118,1.0,42.0
49247,0.0,3.0,1.0,1511335304.0,Internet Explorer,Google Merchandise Store,(not provided),Region_1,Top,(not set),...,/,desktop,Organic Search,Europe,False,1511335304,52759,20171121,1.0,1.0
64879,0.0,2.0,46.0,1470160160.0,Chrome,Google Merchandise Store,(not provided),Region_5,Top,(not set),...,/,desktop,Referral,Americas,False,1470160160,27636,20160802,1.0,30.0
102398,0.0,5.0,54.0,1501561533.0,Chrome,Google Merchandise Store,(not provided),Region_3,Top,(not set),...,/,mobile,Direct,Americas,True,1501561533,113810,20170731,1.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76820,0.0,1.0,1.0,1488284635.0,Firefox,Google Merchandise Store,(not provided),Region_5,Top,(not set),...,/,desktop,Organic Search,Europe,False,1488284635,118810,20170228,1.0,1.0
110268,0.0,1.0,1.0,1522228683.0,Safari (in-app),Google Merchandise Store,(not provided),Region_4,Top,(not set),...,/,mobile,Social,Europe,True,1522228683,45701,20180328,1.0,1.0
103694,0.0,1.0,1.0,1491336808.0,Safari (in-app),Google Merchandise Store,(not provided),Region_1,Top,(not set),...,/xud9jul6em,mobile,Social,Americas,True,1491336808,121727,20170404,1.0,1.0
860,0.0,1.0,59.0,1480877949.0,Chrome,Google Merchandise Store,(not provided),Region_2,Top,(not set),...,/,desktop,Referral,Americas,False,1480877949,6006,20161204,1.0,41.0


## Preprocessing Techniques
### Ordinally Encoding categorical variables and Scaling all features (after encoding categorical variables)

### Using MinMaxScaler() as scaling technique

In [ ]:
ct1 = Pipeline(steps=[
    ("1", OrdinalEncoder()),
    ("2", MinMaxScaler())])
ct2 = Pipeline(steps=[
    ("1", MinMaxScaler())])
pipe = ColumnTransformer([
    ("A", ct1, cat1),
    ("B", ct2, num1)
])

### Using RobustScaler() as scaling technique

In [14]:
ct3 = Pipeline(steps=[
    ("1", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)),
    ("2", RobustScaler())])
ct4 = Pipeline(steps=[
    ("1", RobustScaler())])
pipe1 = ColumnTransformer([
    ("A", ct3, cat1),
    ("B", ct4, num1)
])

In [85]:
ct3 = Pipeline(steps=[
    ("1", OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1))
    #,("2", StandardScaler())
]
    )
ct4 = Pipeline(steps=[
    ("1", StandardScaler())])
pipe1 = ColumnTransformer([
    ("A", ct3, cat1),
    ("B", ct4, num1)
])

In [86]:
x1 = pipe1.fit_transform(x_train)
c = pipe1.get_feature_names_out()
#print(c)
x1.shape

(92818, 29)

In [87]:
x12 = pipe1.transform(x_test)
x12.shape

(23205, 29)

In [88]:
poly = PolynomialFeatures(degree=2, include_bias=False)
x1poly = poly.fit_transform(x1)
xsel = x1poly

In [89]:
xsel.shape

(92818, 464)

In [ ]:
select = SelectKBest(f_regression, k=29)
xsel = select.fit_transform(x1, y_train)
xsel.shape

In [27]:
pca = PCA(n_components=20)
xpca = pca.fit_transform(xsel)
xpca

array([[-6.68358307e+04, -9.28144355e+02, -1.32820586e+02, ...,
         4.37448716e-01, -2.96825902e+00,  1.98614661e+00],
       [-6.68361694e+04, -9.29432909e+02,  9.58035662e+02, ...,
         5.93564285e-01, -1.28447537e+00,  2.37242519e+00],
       [-6.68358317e+04, -9.28148049e+02, -1.28783049e+02, ...,
         4.45658065e-01, -2.91582309e+00,  2.01294751e+00],
       ...,
       [ 2.84836671e+05, -1.12343385e+02, -2.71686648e+01, ...,
         6.92065290e+02,  4.98529955e-01, -6.00851961e+00],
       [-6.68358313e+04, -9.28145880e+02, -1.32813481e+02, ...,
         1.56490783e-01, -2.80854901e+00,  2.13242764e+00],
       [-6.68358307e+04, -9.28144393e+02, -1.32820613e+02, ...,
         4.34689593e-01, -2.96935530e+00,  1.98100406e+00]])

In [43]:
model = Lasso(alpha=1, tol=0.8)
model.fit(xsel, (y_train))
xtsel = poly.transform(x12)
yp = model.predict(xtsel)
yp1 = np.where(yp > 0, yp, 0)
r2_score(y_test, yp1)

0.20740731107436505

In [ ]:
param = {
    "learning_rate" : [0.1, 0.2, 0.24, 0.3, 0.8, 1],
    "max_iter" : [50, 100, 200, 400],
    "max_leaf_nodes" : [20, 31, 25, 35, 45],
    "min_samples_leaf" : [15, 20, 25, 30],
    "l2_regularization":[0, 0.1, 0.5, 1]
}
gcv = GridSearchCV(estimator=HistGradientBoostingRegressor(),
                  param_grid=param,
                  cv=5,
                  scoring="r2",
                  n_jobs=-1)
gcv.fit(xsel, y_train)
gcv.best_params_

In [100]:
model = HistGradientBoostingRegressor(learning_rate=0.2, max_iter=100)
model.fit(xsel, (y_train))
xtsel = poly.transform(x12)
yp = model.predict(xtsel)
yp1 = np.where(yp > 0, yp, 0)
r2_score(y_test, yp1)

0.2631802683740242

In [31]:
model = Lasso(alpha=1, tol=0.8)
model.fit(xpca, y_train)
xtsel = poly.transform(x12)
model.score(pca.transform(xtsel), y_test)

0.08577605037661973

In [ ]:
model = Ridge(alpha=100100, tol=10000, solver="cholesky")
model.fit(xsel, y_train)
xtsel = poly.transform(x12)
yp = model.predict(xtsel)
r2_score(y_test, yp)

In [ ]:
model = SGDRegressor(penalty="l1", loss="huber")
model.fit(xsel, y_train)
xtsel = poly.transform(x12)
yp = model.predict(xtsel)
r2_score(y_test, yp)

In [101]:
X_test = pd.read_csv("/kaggle/input/engage-2-value-from-clicks-to-conversions/test_data.csv") 
xd = pipe_impute.transform(X_test)
x2t = pd.DataFrame(xd, columns=num1+cat1)
xt1 = pipe1.transform(x2t)
xt2 = poly.transform(xt1)
y_pred = model.predict(xt2)
y_pred1 = np.where(y_pred > 0, y_pred, 0)
submission = pd.DataFrame({"id": range(0,X_test.shape[0]), "purchaseValue": y_pred1}) 
submission.to_csv('submission.csv',index=False)

### Scaling Target variables for better result

In [ ]:
tar_sc = MinMaxScaler()
y1 = tar_sc.fit_transform(np.array(y_train).reshape(-1,1))
y1

In [ ]:
y12 = tar_sc.fit_transform(np.array(y_test).reshape(-1,1))
y12

### Fitting a model and testing it, by evaluating r2 score 

In [ ]:
model = MLPRegressor()
model.fit(x1, y1)
y_pred = model.predict(x12)
print(r2_score(y_pred, y12))

## Error in REGRESSION Approach: Negative R2 score and 80% entries in data having 0 value, thus data being highly skewed towards 0
### As noticed in the below cell, nearly 80% of the entries in the target variable has the value 0, but other 20% entries, have significantly higher value
### As the dataset is about E-Commerce industry, having 0 value in purchaseValue is acceptable as all people viewing the website might not buy a product.
### So, those 80 % entries cannot be removed
## New Approach: 
### First performing BINARY CLASSIFICATION of data, classifying the test variable as the entry has made a purchase or not
### If entry is made, then Regression is performed to find the purchaseValue, if not purchaseValue is kept at 0

In [ ]:
print(x2.shape[0] == y.shape[0])

### Splitting the dataset, into classification and regression data set, y_cal referring to series of purchaseMade variable having 0 and 1 values.
### y_reg having series of data, having purchaseValue and entries which ahve purchaseMade == 1


In [ ]:
def reg_clas_split(x, y):
    x3 = x
    x3["purchaseValue"] = y
    x3["purchaseMade"] = (x3["purchaseValue"] > 0).astype(int)
    y_clas = x3["purchaseMade"]
    y_reg = x3.loc[x3["purchaseMade"] == 1, "purchaseValue"]
    x_regr = x[y_clas == 1]
    x_clas = x3.drop(columns=["purchaseValue", "purchaseMade"])
    x_regr = x_regr.drop(columns=["purchaseValue", "purchaseMade"])
    return (x3, y_clas, x_regr, y_reg)
    
    

In [ ]:
x3 = x2
x3["purchaseValue"] = y
x3

In [ ]:
x3["purchaseMade"] = (x3["purchaseValue"] > 0).astype(int)
y_clas = x3["purchaseMade"]
y_reg = x3.loc[x3["purchaseMade"] == 1, "purchaseValue"]

In [ ]:
y_clas

In [ ]:
x_clas = x2
x_regr = x2[y_clas == 1]
x_clas = x_clas.drop(columns=["purchaseValue", "purchaseMade"])
x_regr = x_regr.drop(columns=["purchaseValue", "purchaseMade"])
print("Shape of Regression Dataset: ", x_regr.shape)
print("Shape of Classification Dataset: ", x_clas.shape)

In [ ]:
target_sc = RobustScaler()

In [ ]:
x_train1 = pipe1.fit_transform(x_train)
x_test1 = pipe1.transform(x_test)

In [ ]:
model_clas = GaussianNB()
model_clas.fit(x_train1, y_train)

In [ ]:
xtrc, ytrc, xtrr, ytrr = reg_clas_split(x_train, y_train)

In [ ]:
model_regr = SGDRegressor(random_state=42)
model_regr.fit(pipe1.transform(xtrr), ytrr)

In [ ]:
y_pred = model_clas.predict_proba(x_test1)
y_pred

In [ ]:
y_pred1 = np.where(y_pred >= 0.2, True, False)
y_pred1

In [ ]:
model_clas.score(x_test_clas1, y_test_clas)

In [ ]:
x_test_for_reg = x_test_clas1[y_pred1==1]
y_test_for_re = y.loc[y_test_clas.index]
y_test_for_reg = y_test_for_re[y_pred1==1]
y_test_for_reg

In [ ]:
y_pred_purchase = np.zeros(y_test_clas.shape)
y_pred_purchase1 = np.zeros(y_test_clas.shape)
y_pred_purchase

In [ ]:
y_pred_regr = model_regr.predict(x_test_for_reg)
y_pred_regr_final = target_sc.inverse_transform(y_pred_regr.reshape(-1, 1))
y_pred_regr_final

In [ ]:
y_pred_purchase[y_pred1] = y_pred_regr_final.ravel()
y_pred_purchase

In [ ]:
y_pred_purchase1[y_pred1] = y_test_for_reg
y_pred_purchase1

In [ ]:
r2_score(y_pred_purchase,y_pred_purchase1) 

In [ ]:
y_for_scor = y.loc[x_test_clas.index]
y_for_score = target_sc.fit_transform(np.array(y_for_scor).reshape(-1, 1))

In [ ]:
(y_pred_purchase)

In [ ]:
r2_score(y_for_score, y_pred_purchase)

# Approach - 2

In [ ]:
y1 = np.array(y)
y_clas1 = np.where(y1 != 0, 1, 0)
y_clas1

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x2, y, test_size=0.2, random_state=42)

In [ ]:
y1 = np.array(y_train)
y_clas1 = np.where(y1 != 0, 1, 0)
y_clas1

In [ ]:
pipe1.fit(x_train)
x3 = pipe1.transform(x_train)
x3

In [ ]:
model_clas = GaussianNB()
y_prob = cross_val_predict(estimator=model_clas, X=x3, y=y_clas1, cv=5, method="predict_proba")
y_prob

In [ ]:
x3.shape

In [ ]:
x5 = np.append(x3, np.array(y_prob[:, 1]).reshape(-1, 1), axis=1)
x5

In [ ]:
x5.shape

In [ ]:
model_reg = SGDRegressor(random_state=42)
model_reg.fit(x5, y_train, sample_weight=y_prob[:, 1])

In [ ]:
len(x_train.columns)

In [ ]:
x3t = pipe1.transform(x_test)
x3t

In [ ]:
y1t = np.array(y_test)
y_clas1t = np.where(y1 != 0, 1, 0)
model_clas.fit(x3, y_clas1t)
prob = model_clas.predict_proba(x3t)
x5t = np.append(x3t, np.array(prob[:, 1]).reshape(-1, 1), axis=1)
y5tc = model_reg.predict(x5t)
y5tc

In [ ]:
r2_score(y_test, y5tc)

# Approach - 4

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x2, y, test_size=0.2, random_state=42)

In [ ]:
y1 = np.array(y_train)
y_clas1 = np.where(y1 != 0, 1, 0)
y_clas1

In [ ]:
x_train

In [ ]:
pipe1.fit(x_train)
x3 = pipe1.transform(x_train)
x3.shape

In [ ]:
pca = PCA(n_components=10)
x3_pca = pca.fit_transform(x3)
x3_pca

In [ ]:
x3 = x3_pca
x3.shape

In [ ]:
model_clas = GaussianNB()
y_prob = cross_val_predict(estimator=model_clas, X=x3, y=y_clas1, cv=5, method="predict_proba")
y_prob

In [ ]:
model_clas = SVC(probability=True)
y_prob = cross_val_predict(estimator=model_clas, X=x3, y=y_clas1, cv=5, method="predict_proba")
y_prob

In [ ]:
x5 = np.append(x3, np.array(y_prob[:, 1]).reshape(-1, 1), axis=1)
x5

In [ ]:
x5.shape

In [ ]:
model_reg = LinearRegression()
model_reg.fit(x5, (y_train))

In [ ]:
model_reg = Ridge()
model_reg.fit(x5, (y_train))

In [ ]:
model_reg = Lasso()
model_reg.fit(x5, (y_train), sample_weight=y_prob[:, 1])

In [ ]:
model_reg = MLPRegressor()
model_reg.fit(x5, (y_train))

In [ ]:
model_reg = DecisionTreeRegressor()
model_reg.fit(x5, (y_train))

In [ ]:
model_reg = GradientBoostingRegressor()
model_reg.fit(x5, (y_train))

In [ ]:
x3t = pipe1.transform(x_test)
x3t.shape

In [ ]:
pca = PCA(n_components=10)
x3_pcat = pca.fit_transform(x3t)
x3_pcat

In [ ]:
x3_pcat.shape

In [ ]:
y1t = np.array(y_test)
y_clas1t = np.where(y1 != 0, 1, 0)
model_clas.fit(x3, y_clas1t)
prob = model_clas.predict_proba(x3t)
x5t = np.append(x3t, np.array(prob[:, 1]).reshape(-1, 1), axis=1)
y5tc = model_reg.predict(x5t)
y5tc

In [ ]:
y5tc1 = np.where(y5tc >= 0, y5tc, 0)
y5tc1

In [ ]:
r2_score(y_test, (y5tc1))

In [ ]:
X_test = pd.read_csv("/kaggle/input/engage-2-value-from-clicks-to-conversions/test_data.csv") 
xd = pipe_impute.transform(X_test)
x2t = pd.DataFrame(xd, columns=num1+cat1)
x3t = pipe1.transform(x2t)
#xt2 = select.transform(xt1)
prob = model_clas.predict_proba(x3t)
x5t = np.append(x3t, np.array(prob[:, 1]).reshape(-1, 1), axis=1)
y5tc = model_reg.predict(x5t)
y5tc1 = np.where(y5tc >= 0, y5tc, 0)
#y_pred = model.predict(xt2)
submission = pd.DataFrame({"id": range(0,X_test.shape[0]), "purchaseValue": y5tc1}) 
submission.to_csv('submission.csv',index=False)

In [ ]:
params = {
    "max_depth" : [20, 30],
    "min_samples_split": [2, 5],
    "n_estimators": [50, 100]
}
m = RandomForestRegressor(random_state=42)
gscv = GridSearchCV(estimator=m, param_grid=params, scoring="r2", cv=3, n_jobs=1, verbose=1)
gscv.fit(x2, y1.ravel())
print(gscv.best_params_)

In [ ]:
model = RandomForestRegressor(random_state=42, max_depth=20, min_samples_split=2, n_estimators=50)
model.fit(x2, y1)
r = model.score((pipe.fit_transform(x_test)).toarray(), (ct2.fit_transform(np.array(y_train).reshape(-1, 1)).ravel()))
r

In [ ]:
params = {
    "n_estimators": [10, 20, 30],
    "learning_rate": [3, 9]
}
gscv = GridSearchCV(AdaBoostRegressor(random_state=42), params, scoring="accuracy", cv=3)
gscv.fit(x2, y1)
print(gscv.best_params_)

In [ ]:
x2[x2["device.isMobile"] == True]["pageViews"].corr(df[df["device.isMobile"] == True]["purchaseValue"])

In [ ]:
from sklearn.linear_model import LinearRegression
x3 = x2[["totalHits", "pageViews"]]
x3

m = LinearRegression()
m.fit(x3, y)

pd.Series(m.coef_, index=x3.columns)

In [ ]:
for i in num:
    print(i, x2[i].nunique(), sep=" - ")

# **Milestone 3 - Model Selection**

In [ ]:
`